In [1]:
# import the our code
import sys
sys.path.insert(1, '../../')
sys.path.insert(1, '../')
from sc_preprocessing import sc_preprocess

# general imports
import warnings
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [3]:


#Requires the latest pip
#pip install --upgrade pip

# Current stable release for CPU and GPU
#pip install tensorflow

#import tensorflow as tf
#from tensorflow.keras.utils import to_categorical
from scipy.stats import spearmanr, pearsonr
from collections import Counter

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale


# programming stuff
import time
import os, sys
import pickle
from pathlib import Path

Set up experiment parameters

In [4]:
# number of pseudobulks PER patient
n_train = 1000

### create the domains label 

# first for 10x data
Label_full_10x = np.concatenate([np.full(n_train, 0), np.full(n_train, 1),
                            np.full(n_train, 2), np.full(n_train, 3),
                            np.full(n_train, 4), np.full(n_train, 5),
                            np.full(n_train, 6), np.full(n_train, 7),
                            np.full(n_train, 8), np.full(n_train, 9)], axis=0)
label_full_10x = to_categorical(Label_full_10x)

Label_perturb = np.concatenate([np.full(n_train, 1), np.full(n_train, 0),
                            np.full(n_train, 0), np.full(n_train, 0),
                            np.full(n_train, 0), np.full(n_train, 1),
                            np.full(n_train, 1), np.full(n_train, 1),
                            np.full(n_train, 0), np.full(n_train, 0)], axis=0)
label_perturb = to_categorical(Label_perturb)

"""
# the 11th example is unlabeled is we train only
# using 10x data
unlabeled_10x_only_idx = 9

# now for sm2 data
Label_full_sm2 = np.concatenate([np.full(n_train, 0), np.full(n_train, 1), 
                                np.full(n_train, 2), np.full(n_train, 3),
                                np.full(n_train, 4), np.full(n_train, 5)], axis=0)


# when we train on 10x and sm2 we put them together
Label_both = np.concatenate([Label_full_10x, Label_full_sm2+12])
label_both = to_categorical(Label_both)

# we append an additional binary value to indicate sm2 or 10x
Label_both_dim = np.full(n_train*12, 0)
Label_both_sm2_dim = np.full(n_train*6, 1)
Label_both_dim = np.concatenate([Label_both_dim, Label_both_sm2_dim])
Label_both_dim = to_categorical(Label_both_dim)
"""
label_full_10x

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

Load Data

In [5]:
res_path = f"{os.getcwd()}/../../results/single_cell_data/diva_pbmc/"
bp_res_path = f"{os.getcwd()}/../../results/single_cell_data/bp_pbmc/"
cs_res_path = f"{os.getcwd()}/../../results/single_cell_data/cibersort_pbmc/"
data_path = f"{os.getcwd()}/../../data/single_cell_data/augmented_pbmc_data/"


aug_data_path = "/beevol/home/ivicha/Rotation_project/augmented_data"

cybersort_path = "/beevol/home/ivicha/Rotation_project/cibersort"

data_path = "/beevol/home/ivicha/Rotation_project/raw_matrix/matrix.csv"

scpred_path = "/beevol/home/ivicha/Rotation_project/meta_data/meta.csv"




def read_gene_file(res_path, exp_id):
    gene_file = os.path.join(res_path, f"train-{exp_id}-DIVA_features.pkl")
    gene_path = Path(gene_file)
    gene_df_train = pickle.load( open( gene_path, "rb" ) )
    return gene_df_train


exp_id = "pbmc6k-mono"
pbmc6k_X, pbmc6k_Y, pbmc6k_gene_df = sc_preprocess.read_all_diva_files(data_path, range(10), exp_id)
pbmc6k_X.columns = pbmc6k_gene_df


lab_id = "pbmc6k-mono"
unlab_id = "pbmc6k-mono"
pbmc6k_enc = tf.keras.models.load_model(f"{res_path}/{lab_id}_{unlab_id}_encoder")
pbmc6k_train = read_gene_file(res_path, f"{lab_id}-{unlab_id}")

""" 

exp_id = "pbmc_rep2_10xV2"
pbmc_rep2_10xV2_X, pbmc_rep2_10xV2_Y, pbmc_rep2_10xV2_gene_df = sc_preprocess.read_all_diva_files(data_path, range(10), exp_id)
pbmc_rep2_10xV2_X.columns = pbmc_rep2_10xV2_gene_df


exp_id = "pbmc_rep2_sm2"
pbmc_rep2_sm2_X, pbmc_rep2_sm2_Y, pbmc_rep2_sm2_gene_df = sc_preprocess.read_all_diva_files(data_path, range(6), exp_id)
pbmc_rep2_sm2_X.columns = pbmc_rep2_sm2_gene_df["gene_ids"]


exp_id = "pbmc_rep1_sm2"
pbmc_rep1_sm2_X, pbmc_rep1_sm2_Y, pbmc_rep1_sm2_gene_df = sc_preprocess.read_all_diva_files(data_path, range(6), exp_id)
pbmc_rep1_sm2_X.columns = pbmc_rep1_sm2_gene_df["gene_ids"]


exp_id = "pbmc_rep2_10xV2_sm2_cells"
pbmc_rep2_10xV2_sm2_cells_X, pbmc_rep2_10xV2_sm2_cells_Y, pbmc_rep2_10xV2_sm2_cells_gene_df = sc_preprocess.read_all_diva_files(data_path, range(12), exp_id)
pbmc_rep2_10xV2_sm2_cells_X.columns = pbmc_rep2_10xV2_sm2_cells_gene_df["gene_ids"]


lab_id = "pbmc_rep2_10xV2_sm2_cells"
unlab_id = "pbmc_rep2_sm2"
pbmc_rep2_sm2_enc = tf.keras.models.load_model(f"{res_path}/{lab_id}_{unlab_id}_encoder")
pbmc_rep2_sm2_train = read_gene_file(res_path, f"{lab_id}-{unlab_id}")

lab_id = "pbmc_rep2_10xV2_sm2_cells"
unlab_id = "pbmc_rep2_10xV2_sm2_cells"
pbmc_rep2_10xV2_sm2_cells_enc = tf.keras.models.load_model(f"{res_path}/{lab_id}_{unlab_id}_encoder")
pbmc_rep2_10xV2_sm2_cells_train = read_gene_file(res_path, f"{lab_id}-{unlab_id}")

 """


FileNotFoundError: [Errno 2] No such file or directory: '/Users/ivicha/Documents/VSCode/../../data/single_cell_data/augmented_pbmc_data/pbmc6k-mono_prop_0.pkl'